# SQLite and User Authentication Tutorial
# A beginner's guide to database operations and secure user management


Welcome! In this tutorial, you'll learn:
1. SQLite database basics
2. How to perform CRUD operations (Create, Read, Update, Delete)
3. How to build a secure user authentication system

# ============================================================================
# PART 1: Introduction to SQLite
# ============================================================================


In [16]:
# Necessary modules for this session
import sqlite3
import hashlib
import os
from datetime import datetime

What is SQLite?
SQLite is a lightweight, file-based database that doesn't require a separate server. It's perfect for learning and small applications.

## Creating a connection to the database

In [17]:
conn = sqlite3.connect('tutorial.db')
cursor = conn.cursor()

print("\n✓ Database connection established!")


✓ Database connection established!


# ============================================================================
# PART 2: Creating Your First Table
# ============================================================================


In [18]:
# Let's create a simple practice table first
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER,
        grade TEXT
    )
''')

print("\n✓ 'students' table created!")


✓ 'students' table created!


Key SQL concepts explained:
- PRIMARY KEY: Uniquely identifies each row
- AUTOINCREMENT: Automatically generates sequential IDs
- NOT NULL: Field must have a value
- TEXT, INTEGER: Data types

# ============================================================================
# PART 3: CRUD Operations - The Foundation of Database Work
# ============================================================================


In [19]:
# CREATE - Adding data
print("\n1. CREATE - Adding records:")
students_data = [
    ("Alice Johnson", 20, "A"),
    ("Bob Smith", 22, "B"),
    ("Charlie Brown", 21, "A")
]

cursor.executemany('''
    INSERT INTO students (name, age, grade) VALUES (?, ?, ?)
''', students_data)
conn.commit()
print(f"   ✓ Added {len(students_data)} students")


1. CREATE - Adding records:
   ✓ Added 3 students


In [20]:
# READ - Retrieving data
print("\n2. READ - Fetching records:")
cursor.execute("SELECT * FROM students")
all_students = cursor.fetchall()
for student in all_students:
    print(f"   ID: {student[0]}, Name: {student[1]}, Age: {student[2]}, Grade: {student[3]}")

# UPDATE - Modifying data
print("\n3. UPDATE - Modifying records:")
cursor.execute('''
    UPDATE students SET grade = 'A+' WHERE name = 'Alice Johnson'
''')
conn.commit()
print("   ✓ Updated Alice's grade to A+")


2. READ - Fetching records:
   ID: 1, Name: Alice Johnson, Age: 20, Grade: A+
   ID: 2, Name: Bob Smith, Age: 22, Grade: B
   ID: 4, Name: Alice Johnson, Age: 20, Grade: A
   ID: 5, Name: Bob Smith, Age: 22, Grade: B
   ID: 6, Name: Charlie Brown, Age: 21, Grade: A

3. UPDATE - Modifying records:
   ✓ Updated Alice's grade to A+


In [21]:
# UPDATE - Modifying data
print("\n3. UPDATE - Modifying records:")
cursor.execute('''
    UPDATE students SET grade = 'A+' WHERE name = 'Alice Johnson'
''')
conn.commit()
print("   ✓ Updated Alice's grade to A+")


3. UPDATE - Modifying records:
   ✓ Updated Alice's grade to A+


In [22]:
# DELETE - Removing data
print("\n4. DELETE - Removing records:")
cursor.execute('''
    DELETE FROM students WHERE name = 'Charlie Brown'
''')
conn.commit()
print("   ✓ Removed Charlie from database")


4. DELETE - Removing records:
   ✓ Removed Charlie from database


# ============================================================================
# PART 4: Building a User Authentication System
# ============================================================================


In [23]:
# Step 1: Create the users table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        salt TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')
conn.commit()
print("\n✓ 'users' table created with security fields")


✓ 'users' table created with security fields


Why these fields?
- username: UNIQUE ensures no duplicate usernames
- password_hash: We NEVER store plain passwords!
- salt: Random data added to passwords before hashing (extra security)
- created_at: Timestamp for record keeping

In [24]:
# Step 2: Password Hashing Functions
def generate_salt():
    """Generate a random salt for password hashing"""
    return os.urandom(32).hex()

def hash_password(password, salt):
    """Hash a password with a salt using SHA-256"""
    # Combine password and salt, then hash
    salted_password = (password + salt).encode('utf-8')
    return hashlib.sha256(salted_password).hexdigest()

print("\n✓ Password hashing functions defined")
print("   - Passwords will be hashed using SHA-256")
print("   - Each password gets a unique salt")


✓ Password hashing functions defined
   - Passwords will be hashed using SHA-256
   - Each password gets a unique salt


In [25]:
# Step 3: User Registration Function
def register_user(username, password):
    """
    Register a new user with secure password storage
    
    Args:
        username: Desired username
        password: Plain text password (will be hashed)
    
    Returns:
        bool: True if successful, False if username exists
    """
    try:
        # Generate a unique salt for this user
        salt = generate_salt()
        
        # Hash the password with the salt
        password_hash = hash_password(password, salt)
        
        # Store in database
        cursor.execute('''
            INSERT INTO users (username, password_hash, salt)
            VALUES (?, ?, ?)
        ''', (username, password_hash, salt))
        conn.commit()
        
        print(f"   ✓ User '{username}' registered successfully!")
        return True
        
    except sqlite3.IntegrityError:
        # This happens if username already exists
        print(f"   ✗ Username '{username}' already exists!")
        return False

In [26]:
# Step 4: User Login Function
def login_user(username, password):
    """
    Authenticate a user by verifying their password
    
    Args:
        username: Username to authenticate
        password: Plain text password to verify
    
    Returns:
        bool: True if authentication successful, False otherwise
    """
    # Retrieve user data
    cursor.execute('''
        SELECT password_hash, salt FROM users WHERE username = ?
    ''', (username,))
    
    result = cursor.fetchone()
    
    if result is None:
        print(f"   ✗ User '{username}' not found!")
        return False
    
    stored_hash, salt = result
    
    # Hash the provided password with the stored salt
    input_hash = hash_password(password, salt)
    
    # Compare hashes
    if input_hash == stored_hash:
        print(f"   ✓ Login successful! Welcome, {username}!")
        return True
    else:
        print(f"   ✗ Incorrect password for '{username}'")
        return False

In [27]:
# Step 5: Additional Helper Functions
def user_exists(username):
    """Check if a username exists in the database"""
    cursor.execute('SELECT id FROM users WHERE username = ?', (username,))
    return cursor.fetchone() is not None

def change_password(username, old_password, new_password):
    """Change a user's password after verifying the old one"""
    # First, verify the old password
    cursor.execute('''
        SELECT password_hash, salt FROM users WHERE username = ?
    ''', (username,))
    
    result = cursor.fetchone()
    if result is None:
        print(f"   ✗ User '{username}' not found!")
        return False
    
    stored_hash, old_salt = result
    input_hash = hash_password(old_password, old_salt)
    
    if input_hash != stored_hash:
        print(f"   ✗ Current password is incorrect!")
        return False
    
    # Generate new salt and hash for new password
    new_salt = generate_salt()
    new_hash = hash_password(new_password, new_salt)
    
    # Update database
    cursor.execute('''
        UPDATE users SET password_hash = ?, salt = ?
        WHERE username = ?
    ''', (new_hash, new_salt, username))
    conn.commit()
    
    print(f"   ✓ Password changed successfully for '{username}'!")
    return True

def delete_user(username):
    """Delete a user from the database"""
    cursor.execute('DELETE FROM users WHERE username = ?', (username,))
    conn.commit()
    print(f"   ✓ User '{username}' deleted successfully!")

# ============================================================================
# PART 5: Testing the Authentication System
# ============================================================================


In [28]:
# Test 1: Register new users
print("\nTest 1: Registering new users")
register_user("alice", "securePassword123")
register_user("bob", "bobsPassword456")
register_user("alice", "differentPassword")  # Should fail - duplicate

# Test 2: Login attempts
print("\nTest 2: Login attempts")
login_user("alice", "securePassword123")  # Should succeed
login_user("alice", "wrongPassword")      # Should fail
login_user("charlie", "anyPassword")      # Should fail - user doesn't exist

# Test 3: Change password
print("\nTest 3: Changing password")
change_password("alice", "securePassword123", "newPassword789")
login_user("alice", "newPassword789")  # Should succeed with new password

# Test 4: View all users (without showing passwords!)
print("\nTest 4: View registered users")
cursor.execute('SELECT id, username, created_at FROM users')
all_users = cursor.fetchall()
print(f"   Total registered users: {len(all_users)}")
for user in all_users:
    print(f"   - ID: {user[0]}, Username: {user[1]}, Created: {user[2]}")



Test 1: Registering new users
   ✗ Username 'alice' already exists!
   ✓ User 'bob' registered successfully!
   ✗ Username 'alice' already exists!

Test 2: Login attempts
   ✓ Login successful! Welcome, alice!
   ✗ Incorrect password for 'alice'
   ✗ User 'charlie' not found!

Test 3: Changing password
   ✓ Password changed successfully for 'alice'!
   ✓ Login successful! Welcome, alice!

Test 4: View registered users
   Total registered users: 2
   - ID: 1, Username: alice, Created: 2025-10-18 14:50:24
   - ID: 2, Username: bob, Created: 2025-10-18 14:54:51


# ============================================================================
# PART 6: Best Practices and Security Tips
# ============================================================================


In [29]:
security_tips = """
✓ DO's:
  1. Always hash passwords - NEVER store plain text passwords
  2. Use a unique salt for each user
  3. Use prepared statements (?) to prevent SQL injection
  4. Validate input before processing
  5. Use strong hashing algorithms (SHA-256 or better, bcrypt is even better)
  6. Implement password strength requirements
  7. Consider adding rate limiting for login attempts
  8. Use HTTPS in production environments

✗ DON'Ts:
  1. Never store passwords in plain text
  2. Don't use the same salt for all users
  3. Don't concatenate user input directly into SQL queries
  4. Don't ignore error handling
  5. Don't log or display actual passwords

For Production Systems, Consider:
  - Using bcrypt or argon2 instead of SHA-256 (they're designed for passwords)
  - Adding two-factor authentication (2FA)
  - Implementing account lockout after failed attempts
  - Adding password reset functionality
  - Using environment variables for sensitive configuration
"""

print(security_tips)


✓ DO's:
  1. Always hash passwords - NEVER store plain text passwords
  2. Use a unique salt for each user
  3. Use prepared statements (?) to prevent SQL injection
  4. Validate input before processing
  5. Use strong hashing algorithms (SHA-256 or better, bcrypt is even better)
  6. Implement password strength requirements
  7. Consider adding rate limiting for login attempts
  8. Use HTTPS in production environments

✗ DON'Ts:
  1. Never store passwords in plain text
  2. Don't use the same salt for all users
  3. Don't concatenate user input directly into SQL queries
  4. Don't ignore error handling
  5. Don't log or display actual passwords

For Production Systems, Consider:
  - Using bcrypt or argon2 instead of SHA-256 (they're designed for passwords)
  - Adding two-factor authentication (2FA)
  - Implementing account lockout after failed attempts
  - Adding password reset functionality
  - Using environment variables for sensitive configuration



# ============================================================================
# PART 7: Interactive Demo 
# ============================================================================


In [30]:
def interactive_demo():
    """Simple interactive menu for testing the system"""
    while True:
        print("\n--- User Authentication Menu ---")
        print("1. Register new user")
        print("2. Login")
        print("3. Change password")
        print("4. View all users")
        print("5. Exit")
        
        choice = input("\nEnter your choice (1-5): ").strip()
        
        if choice == "1":
            username = input("Enter username: ").strip()
            password = input("Enter password: ").strip()
            register_user(username, password)
            
        elif choice == "2":
            username = input("Enter username: ").strip()
            password = input("Enter password: ").strip()
            login_user(username, password)
            
        elif choice == "3":
            username = input("Enter username: ").strip()
            old_pass = input("Enter current password: ").strip()
            new_pass = input("Enter new password: ").strip()
            change_password(username, old_pass, new_pass)
            
        elif choice == "4":
            cursor.execute('SELECT id, username, created_at FROM users')
            users = cursor.fetchall()
            print(f"\nTotal users: {len(users)}")
            for user in users:
                print(f"  - {user[1]} (ID: {user[0]}, Created: {user[2]})")
                
        elif choice == "5":
            print("\nGoodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

#interactive_demo()


# ============================================================================
# Cleanup and Close Connection
# ============================================================================


In [31]:
# Always close your database connection when done
conn.close()
print("\n✓ Database connection closed")


✓ Database connection closed
